In [1]:
from typing import Union
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from pathlib import Path
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import SGDOneClassSVM
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler

In [2]:
RANDOM_SEED = 42
DATA_PATH = Path("../data")

np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)


In [3]:
train_data = pd.read_csv(DATA_PATH / "train.csv")
test_data = pd.read_csv(DATA_PATH / "test.csv")

In [4]:
print(train_data['xmeas_39'].min(),train_data['xmeas_39'].max()) #0.06~0.14
print(train_data['xmeas_40'].min(),train_data['xmeas_40'].max()) #51~56
print(train_data['xmeas_41'].min(),train_data['xmeas_41'].max()) #41~56
print(train_data['xmeas_14'].min(),train_data['xmeas_14'].max()) #20~30
print(train_data['xmeas_5'].min(),train_data['xmeas_5'].max()) #25~28
print(train_data['xmeas_6'].min(),train_data['xmeas_6'].max()) #41~44




0.060641 0.13718
51.564 55.773
41.768 45.979
20.752 29.855
25.951 27.818
41.394 43.257


In [4]:
train_data.columns

Index(['faultNumber', 'simulationRun', 'sample', 'xmeas_1', 'xmeas_2',
       'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8',
       'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14',
       'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20',
       'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26',
       'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32',
       'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38',
       'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4',
       'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11'],
      dtype='object')

In [5]:
def process_data(df) -> pd.DataFrame:
    numeric_cols = [
       'xmeas_1', 'xmeas_2',
       'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8',
       'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14',
       'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20',
       'xmeas_21', 'xmeas_22', 'xmeas_23', 'xmeas_24', 'xmeas_25', 'xmeas_26',
       'xmeas_27', 'xmeas_28', 'xmeas_29', 'xmeas_30', 'xmeas_31', 'xmeas_32',
       'xmeas_33', 'xmeas_34', 'xmeas_35', 'xmeas_36', 'xmeas_37', 'xmeas_38',
       'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4',
       'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11'
    ]
    return df[numeric_cols]

In [6]:
train_df = process_data(train_data)
test_df = process_data(test_data)


In [19]:
class HybridAnomalyDetector:
    def __init__(self):
        self.iso_forest = IsolationForest(contamination=0.05, random_state=42)
        self.sgd_svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.05)
        self.scaler = StandardScaler()
        self.feature_stats = {}  # 범위 기준 저장

    def fit(self, data: pd.DataFrame):
        # 1. 상관계수 계산
        corr_matrix = data.corr()
        high_corr_features = [col for col in data.columns if corr_matrix[col].abs().max() > 0.8]

        # 2. 범위 기반 이상 탐지 기준 저장 (평균 ± 3표준편차)
        for feature in high_corr_features:
            mean = data[feature].mean()
            std = data[feature].std()
            self.feature_stats[feature] = (mean, std)

        # 3. 범위 기반 피처 제거 후 나머지로 모델 학습
        data_for_model = data.drop(columns=high_corr_features)
        scaled_data = self.scaler.fit_transform(data_for_model)

        # IsolationForest와 SGDOneClassSVM 학습
        self.iso_forest.fit(scaled_data)
        self.sgd_svm.fit(scaled_data)

    def predict(self, data: pd.DataFrame) -> pd.DataFrame:
        # 1. 범위 기반 예측 수행
        range_outliers = pd.DataFrame(index=data.index)
        for feature, (mean, std) in self.feature_stats.items():
            lower_bound = mean - 3 * std
            upper_bound = mean + 3 * std
            range_outliers[feature + '_Outlier'] = (~data[feature].between(lower_bound, upper_bound)).astype(int)

        # 2. 나머지 피처에 대해 IsolationForest와 SGDOneClassSVM 적용
        data_for_model = data.drop(columns=self.feature_stats.keys(), errors='ignore')
        scaled_data = self.scaler.transform(data_for_model)

        iso_labels = (self.iso_forest.predict(scaled_data) == -1).astype(int)  # 1: 이상, 0: 정상
        svm_labels = (self.sgd_svm.predict(scaled_data) == -1).astype(int)  # 1: 이상, 0: 정상

        # 3. 결과 통합 (다수결 방식)
        results = pd.DataFrame({
            'IsolationForest': iso_labels,
            'SGDOneClassSVM': svm_labels
        }, index=data.index)

        results = pd.concat([results, range_outliers], axis=1)
        results['Final_Label'] = (results.mean(axis=1) >= 0.5).astype(int)  # 다수결

        return results[['Final_Label']]

In [21]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler

class HybridAnomalyDetector:
    def __init__(self):
        self.iso_forest = IsolationForest(contamination=0.05, random_state=42)
        self.sgd_svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.05)
        self.scaler = StandardScaler()
        self.feature_stats = {}  # 범위 기준 저장
        self.removed_features = []  # 제거된 피처 저장

    def fit(self, data: pd.DataFrame):
        # 1. 상관계수 계산
        corr_matrix = data.corr().abs()  # 상관계수 절대값 사용
        upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1) > 0)  # 상삼각 행렬 추출

        # 2. 상관계수가 0.9 이상인 피처 중 하나를 제거
        self.removed_features = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]

        print(f"제거된 피처: {self.removed_features}")

        # 3. 범위 기반 이상 탐지 기준 저장 (평균 ± 3표준편차)
        for feature in self.removed_features:
            mean = data[feature].mean()
            std = data[feature].std()
            self.feature_stats[feature] = (mean, std)

        # 4. 제거된 피처를 제외하고 나머지 데이터로 모델 학습
        data_for_model = data.drop(columns=self.removed_features, errors='ignore')
        scaled_data = self.scaler.fit_transform(data_for_model)

        # IsolationForest와 One-Class SVM 학습
        self.iso_forest.fit(scaled_data)
        self.sgd_svm.fit(scaled_data)

    def predict(self, data: pd.DataFrame) -> pd.DataFrame:
        # 1. 범위 기반 예측 수행
        range_outliers = pd.DataFrame(index=data.index)
        for feature, (mean, std) in self.feature_stats.items():
            lower_bound = mean - 3 * std
            upper_bound = mean + 3 * std
            range_outliers[feature + '_Outlier'] = (~data[feature].between(lower_bound, upper_bound)).astype(int)

        # 2. 나머지 피처에 대해 IsolationForest와 SGDOneClassSVM 적용
        data_for_model = data.drop(columns=self.removed_features, errors='ignore')
        scaled_data = self.scaler.transform(data_for_model)

        iso_labels = (self.iso_forest.predict(scaled_data) == -1).astype(int)  # 1: 이상, 0: 정상
        svm_labels = (self.sgd_svm.predict(scaled_data) == -1).astype(int)  # 1: 이상, 0: 정상

        # 3. 결과 통합 (다수결 방식)
        results = pd.DataFrame({
            'IsolationForest': iso_labels,
            'SGDOneClassSVM': svm_labels
        }, index=data.index)

        results = pd.concat([results, range_outliers], axis=1)
        results['Final_Label'] = (results.mean(axis=1) >= 0.5).astype(int)  # 다수결

        return results[['Final_Label']]



In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler

class HybridAnomalyDetector:
    def __init__(self):
        self.iso_forest = IsolationForest(contamination=0.05, random_state=42)
        self.sgd_svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.05)
        self.scaler = StandardScaler()
        self.feature_stats = {}  # 범위 기준 저장
        self.removed_features = []  # 제거된 피처 저장

    def fit(self, data: pd.DataFrame):
        # 1. 상관계수 계산
        corr_matrix = data.corr().abs()  # 상관계수 절대값 사용
        upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1) > 0)  # 상삼각 행렬 추출

        # 2. 상관계수가 0.9 이상인 모든 피처 찾기
        high_corr_features = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]

        # 3. 제거된 피처 저장
        self.removed_features = high_corr_features
        print(f"제거된 피처: {self.removed_features}")

        # 4. 범위 기반 기준 저장 (평균 ± 3표준편차)
        for feature in self.removed_features:
            mean = data[feature].mean()
            std = data[feature].std()
            self.feature_stats[feature] = (mean, std)

        # 5. 제거된 피처를 제외하고 나머지 데이터로 모델 학습
        data_for_model = data.drop(columns=self.removed_features, errors='ignore')
        scaled_data = self.scaler.fit_transform(data_for_model)

        # IsolationForest와 One-Class SVM 학습
        self.iso_forest.fit(scaled_data)
        self.sgd_svm.fit(scaled_data)

    def predict(self, data: pd.DataFrame) -> pd.DataFrame:
        # 1. 범위 기반 예측 수행
        range_outliers = pd.DataFrame(index=data.index)
        for feature, (mean, std) in self.feature_stats.items():
            lower_bound = mean - 3 * std
            upper_bound = mean + 3 * std
            range_outliers[feature + '_Outlier'] = (~data[feature].between(lower_bound, upper_bound)).astype(int)

        # 2. 나머지 피처에 대해 IsolationForest와 SGDOneClassSVM 적용
        data_for_model = data.drop(columns=self.removed_features, errors='ignore')
        scaled_data = self.scaler.transform(data_for_model)

        iso_labels = (self.iso_forest.predict(scaled_data) == -1).astype(int)  # 1: 이상, 0: 정상
        svm_labels = (self.sgd_svm.predict(scaled_data) == -1).astype(int)  # 1: 이상, 0: 정상

        # 3. 결과 통합 (다수결 방식)
        results = pd.DataFrame({
            'IsolationForest': iso_labels,
            'SGDOneClassSVM': svm_labels
        }, index=data.index)

        results = pd.concat([results, range_outliers], axis=1)
        results['Final_Label'] = (results.mean(axis=1) >= 0.5).astype(int)  # 다수결

        return results[['Final_Label']]


In [12]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler

class HybridAnomalyDetector:
    def __init__(self):
        self.iso_forest = IsolationForest(contamination=0.05, random_state=42)
        self.sgd_svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.05)
        self.scaler = StandardScaler()
        self.feature_stats = {}  # 범위 기준 저장
        self.removed_features = []  # 제거된 피처 목록

    def fit(self, data: pd.DataFrame):
        # 1. 상관계수 계산
        corr_matrix = data.corr().abs()  # 상관계수 절대값 사용
        upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1) > 0)  # 상삼각 행렬

        # 2. 상관계수가 0.9 이상인 모든 피처 찾기
        self.removed_features = [col for col in upper_tri.columns if any(upper_tri[col] > 0.9)]
        print(f"제거된 피처: {self.removed_features}")

        # 3. 범위 기반 기준 저장 (제거된 피처만 사용, 평균 ± 3표준편차)
        for feature in self.removed_features:
            mean = data[feature].mean()
            std = data[feature].std()
            self.feature_stats[feature] = (mean, std)

        # 4. 제거된 피처를 제외한 데이터로 모델 학습
        data_for_model = data.drop(columns=self.removed_features, errors='ignore')
        scaled_data = self.scaler.fit_transform(data_for_model)

        # IsolationForest와 One-Class SVM 학습
        self.iso_forest.fit(scaled_data)
        self.sgd_svm.fit(scaled_data)

    def predict(self, data: pd.DataFrame) -> pd.DataFrame:
        # 1. 범위 기반 예측 수행 (제거된 피처만 사용)
        range_outliers = pd.DataFrame(index=data.index)
        for feature, (mean, std) in self.feature_stats.items():
            lower_bound = mean - 3 * std
            upper_bound = mean + 3 * std
            range_outliers[feature + '_Outlier'] = (~data[feature].between(lower_bound, upper_bound)).astype(int)

        # 2. 나머지 피처에 대해 IsolationForest와 SGDOneClassSVM 적용
        data_for_model = data.drop(columns=self.removed_features, errors='ignore')
        scaled_data = self.scaler.transform(data_for_model)

        iso_labels = (self.iso_forest.predict(scaled_data) == -1).astype(int)  # 1: 이상, 0: 정상
        svm_labels = (self.sgd_svm.predict(scaled_data) == -1).astype(int)  # 1: 이상, 0: 정상

        # 3. 결과 통합 (다수결 방식)
        results = pd.DataFrame({
            'IsolationForest': iso_labels,
            'SGDOneClassSVM': svm_labels
        }, index=data.index)

        results = pd.concat([results, range_outliers], axis=1)
        results['Final_Label'] = (results.mean(axis=1) >= 0.5).astype(int)  # 다수결

        return results[['Final_Label']]


In [5]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

class HybridAnomalyDetector:
    def __init__(self):
        self.iso_forest = IsolationForest(contamination=0.05, random_state=42)
        self.sgd_svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.05)
        self.scaler = StandardScaler()
        self.feature_stats = {}  # 범위 기준 저장
        self.removed_features = []  # 상관관계 높은 피처 목록 저장
        self.high_corr_groups = []  # 상관관계 높은 피처들 간 그룹 저장
        self.pca_models = {}  # 상관관계 높은 피처 그룹별 PCA 모델 저장

    def fit(self, data: pd.DataFrame):
        # 1. 상관계수 계산
        corr_matrix = data.corr().abs()  # 상관계수 절대값 사용
        upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1) > 0)  # 상삼각 행렬

        # 2. 상관계수가 0.9 이상인 모든 피처 그룹 추출
        self.removed_features = [col for col in upper_tri.columns if any(upper_tri[col] > 0.9)]
        print(f"제거된 피처: {self.removed_features}")

        # 상관관계가 높은 피처들끼리 그룹핑
        for col in self.removed_features:
            correlated_features = list(upper_tri.index[upper_tri[col] > 0.9])
            if correlated_features:
                self.high_corr_groups.append(correlated_features)

        print(f"상관관계 높은 피처 그룹: {self.high_corr_groups}")

        # 3. 각 상관관계 그룹에 대해 범위 기반 기준 저장 (평균 ± 3표준편차)
        for group in self.high_corr_groups:
            for feature in group:
                mean = data[feature].mean()
                std = data[feature].std()
                self.feature_stats[feature] = (mean, std)

            # PCA 모델을 각 그룹에 대해 생성하고, 첫 번째 주성분만 사용할 수 있도록 맞춤
            pca = PCA(n_components=1)
            pca.fit(data[group])
            self.pca_models[tuple(group)] = pca

        # 4. 상관관계 높은 피처들을 제외하고 나머지 데이터로 모델 학습
        data_for_model = data.drop(columns=self.removed_features, errors='ignore')
        scaled_data = self.scaler.fit_transform(data_for_model)

        # IsolationForest와 One-Class SVM 학습
        self.iso_forest.fit(scaled_data)
        self.sgd_svm.fit(scaled_data)

    def predict(self, data: pd.DataFrame) -> pd.DataFrame:
        # 1. 상관관계 높은 피처 그룹에 대해 범위 기반 이상 탐지 수행
        range_outliers = pd.DataFrame(index=data.index)
        for feature, (mean, std) in self.feature_stats.items():
            lower_bound = mean - 3 * std
            upper_bound = mean + 3 * std
            range_outliers[feature + '_Outlier'] = (~data[feature].between(lower_bound, upper_bound)).astype(int)

        # PCA 기반 이상 탐지 수행
        pca_outliers = pd.DataFrame(index=data.index)
        for group, pca in self.pca_models.items():
            group_data = data[list(group)]
            pca_transformed = pca.transform(group_data)
            outlier_label = (np.abs(pca_transformed) > 3).astype(int).ravel()  # 3 표준편차 이상이면 이상치로 설정
            pca_outliers[str(group) + '_PCA_Outlier'] = outlier_label

        # 2. 나머지 피처에 대해 IsolationForest와 One-Class SVM 적용
        data_for_model = data.drop(columns=self.removed_features, errors='ignore')
        scaled_data = self.scaler.transform(data_for_model)

        iso_labels = (self.iso_forest.predict(scaled_data) == -1).astype(int)  # 1: 이상, 0: 정상
        svm_labels = (self.sgd_svm.predict(scaled_data) == -1).astype(int)  # 1: 이상, 0: 정상

        # 3. 결과 통합 (다수결 방식)
        results = pd.DataFrame({
            'IsolationForest': iso_labels,
            'SGDOneClassSVM': svm_labels
        }, index=data.index)

        # 상관관계 기반 이상 탐지 결과 추가
        results = pd.concat([results, range_outliers, pca_outliers], axis=1)

        # 최종 다수결: 모든 탐지 결과의 평균이 0.5 이상이면 이상치로 판별
        results['Final_Label'] = (results.mean(axis=1) >= 0.5).astype(int)

        return results[['Final_Label']]


In [7]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

class HybridAnomalyDetector:
    def __init__(self):
        self.iso_forest = IsolationForest(contamination=0.05, random_state=42)
        self.sgd_svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.05)
        self.scaler = StandardScaler()
        self.feature_stats = {}  # 범위 기준 저장
        self.removed_features = []  # 상관관계 높은 피처 목록 저장
        self.high_corr_groups = []  # 상관관계 높은 피처들 간 그룹 저장
        self.low_corr_features = []  # 상관관계가 낮은 피처 저장 (상관계수 < 0.2)
        self.pca_models = {}  # 상관관계 높은 피처 그룹별 PCA 모델 저장

    def fit(self, data: pd.DataFrame):
        # 1. 상관계수 계산
        corr_matrix = data.corr().abs()  # 상관계수 절대값 사용
        upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1) > 0)  # 상삼각 행렬

        # 2. 상관계수가 0.9 이상인 모든 피처 그룹 추출
        self.removed_features = [col for col in upper_tri.columns if any(upper_tri[col] > 0.9)]
        print(f"제거된 피처: {self.removed_features}")

        # 상관관계가 높은 피처들끼리 그룹핑
        for col in self.removed_features:
            correlated_features = list(upper_tri.index[upper_tri[col] > 0.9])
            if correlated_features:
                self.high_corr_groups.append(correlated_features)

        print(f"상관관계 높은 피처 그룹: {self.high_corr_groups}")

        # 3. 상관계수가 0.2 이하인 피처 목록 추출
        for col in data.columns:
            if all(upper_tri[col] < 0.2):  # 상관계수 0.2 미만인 피처들
                self.low_corr_features.append(col)

        print(f"상관관계가 낮은 피처: {self.low_corr_features}")

        # 4. 상관관계가 낮은 피처들에 대해 범위 기준 저장 (평균 ± 3표준편차)
        for feature in self.low_corr_features:
            mean = data[feature].mean()
            std = data[feature].std()
            self.feature_stats[feature] = (mean, std)

        # 5. 각 상관관계 그룹에 대해 PCA 적용 및 모델 저장
        for group in self.high_corr_groups:
            pca = PCA(n_components=1)
            pca.fit(data[group])
            self.pca_models[tuple(group)] = pca

        # 6. 상관관계 높은 피처들과 상관관계 낮은 피처들을 제외하고 나머지 데이터로 IsolationForest와 SVM 학습
        features_to_remove = self.removed_features + self.low_corr_features
        data_for_model = data.drop(columns=features_to_remove, errors='ignore')
        scaled_data = self.scaler.fit_transform(data_for_model)

        # IsolationForest와 One-Class SVM 학습
        self.iso_forest.fit(scaled_data)
        self.sgd_svm.fit(scaled_data)

    def predict(self, data: pd.DataFrame) -> pd.DataFrame:
        # 1. 상관관계 높은 피처 그룹에 대해 PCA 기반 이상 탐지 수행
        pca_outliers = pd.DataFrame(index=data.index)
        for group, pca in self.pca_models.items():
            group_data = data[list(group)]
            pca_transformed = pca.transform(group_data)
            outlier_label = (np.abs(pca_transformed) > 3).astype(int).ravel()  # 3 표준편차 이상이면 이상치로 설정
            pca_outliers[str(group) + '_PCA_Outlier'] = outlier_label

        # 2. 상관관계가 낮은 피처들에 대해 범위 기반 이상 탐지 수행
        range_outliers = pd.DataFrame(index=data.index)
        for feature, (mean, std) in self.feature_stats.items():
            lower_bound = mean - 3 * std
            upper_bound = mean + 3 * std
            range_outliers[feature + '_Outlier'] = (~data[feature].between(lower_bound, upper_bound)).astype(int)

        # 3. 상관관계가 높은 피처들과 상관관계가 낮은 피처를 제외한 나머지 데이터에 대해 IsolationForest와 One-Class SVM 적용
        features_to_remove = self.removed_features + self.low_corr_features
        data_for_model = data.drop(columns=features_to_remove, errors='ignore')
        scaled_data = self.scaler.transform(scaled_data)

        iso_labels = (self.iso_forest.predict(scaled_data) == -1).astype(int)  # 1: 이상, 0: 정상
        svm_labels = (self.sgd_svm.predict(scaled_data) == -1).astype(int)  # 1: 이상, 0: 정상

        # 4


In [7]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

class HybridAnomalyDetector:
    def __init__(self):
        self.iso_forest = IsolationForest(contamination=0.05, random_state=42)
        self.sgd_svm = OneClassSVM(kernel='rbf', gamma='auto', nu=0.05)
        self.scaler = StandardScaler()
        self.feature_stats = {}  # 범위 기준 저장
        self.removed_features = []  # 상관관계 높은 피처 목록 저장
        self.high_corr_groups = []  # 상관관계 높은 피처들 간 그룹 저장 (상관계수 > 0.9)
        self.low_corr_features = []  # 상관관계가 낮은 피처 저장 (상관계수 < 0.2)
        self.pca_models = {}  # 상관관계 높은 피처 그룹별 PCA 모델 저장

    def fit(self, data: pd.DataFrame):
        # 1. 상관계수 계산
        corr_matrix = data.corr().abs()  # 상관계수 절대값 사용
        upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1) > 0)  # 상삼각 행렬

        # 2. 상관계수가 0.9 이상인 모든 피처 그룹 추출
        self.removed_features = [col for col in upper_tri.columns if any(upper_tri[col] > 0.9)]
        print(f"제거된 피처: {self.removed_features}")

        # 상관관계가 높은 피처들끼리 그룹핑
        for col in self.removed_features:
            correlated_features = list(upper_tri.index[upper_tri[col] > 0.9])
            if correlated_features:
                self.high_corr_groups.append(correlated_features)

        print(f"상관관계 높은 피처 그룹: {self.high_corr_groups}")

        # 3. 상관계수가 0.2 이하인 피처 목록 추출
        # 모든 피처를 탐색하면서 다른 피처들과의 상관계수가 0.2 이하인 경우에만 추가
        for col in data.columns:
            # col 자신을 제외한 다른 피처들과의 상관관계가 모두 0.2 미만인지 확인
            other_corrs = corr_matrix[col].drop(col)  # 자신을 제외
            if all(other_corrs < 0.2):  # 모든 다른 피처와의 상관계수가 0.2 미만일 때만 추가
                self.low_corr_features.append(col)

        print(f"상관관계가 낮은 피처: {self.low_corr_features}")

        # 4. 상관관계가 낮은 피처들에 대해 범위 기준 저장 (평균 ± 3표준편차)
        for feature in self.low_corr_features:
            mean = data[feature].mean()
            std = data[feature].std()
            self.feature_stats[feature] = (mean, std)

        # 5. 각 상관관계 그룹에 대해 PCA 적용 및 모델 저장
        for group in self.high_corr_groups:
            pca = PCA(n_components=1)
            pca.fit(data[group])
            self.pca_models[tuple(group)] = pca

        # 6. 상관관계 높은 피처들과 상관관계 낮은 피처들을 제외하고 나머지 데이터로 IsolationForest와 SVM 학습
        features_to_remove = self.removed_features + self.low_corr_features
        data_for_model = data.drop(columns=features_to_remove, errors='ignore')
        scaled_data = self.scaler.fit_transform(data_for_model)

        # IsolationForest와 One-Class SVM 학습
        self.iso_forest.fit(scaled_data)
        self.sgd_svm.fit(scaled_data)

    def predict(self, data: pd.DataFrame) -> pd.DataFrame:
        # 1. 상관관계 높은 피처 그룹에 대해 PCA 기반 이상 탐지 수행
        pca_outliers = pd.DataFrame(index=data.index)
        for group, pca in self.pca_models.items():
            group_data = data[list(group)]
            pca_transformed = pca.transform(group_data)
            outlier_label = (np.abs(pca_transformed) > 3).astype(int).ravel()  # 3 표준편차 이상이면 이상치로 설정
            pca_outliers[str(group) + '_PCA_Outlier'] = outlier_label

        # 2. 상관관계가 낮은 피처들에 대해 범위 기반 이상 탐지 수행
        range_outliers = pd.DataFrame(index=data.index)
        for feature, (mean, std) in self.feature_stats.items():
            lower_bound = mean - 3 * std
            upper_bound = mean + 3 * std
            range_outliers[feature + '_Outlier'] = (~data[feature].between(lower_bound, upper_bound)).astype(int)

        # 3. 상관관계가 높은 피처들과 상관관계가 낮은 피처를 제외한 나머지 데이터에 대해 IsolationForest와 One-Class SVM 적용
        features_to_remove = self.removed_features + self.low_corr_features
        data_for_model = data.drop(columns=features_to_remove, errors='ignore')
        scaled_data = self.scaler.transform(data_for_model)

        iso_labels = (self.iso_forest.predict(scaled_data) == -1).astype(int)  # 1: 이상, 0: 정상
        svm_labels = (self.sgd_svm.predict(scaled_data) == -1).astype(int)  # 1: 이상, 0: 정상

        # 4. 결과 통합 (다수결 방식)
        results = pd.DataFrame({
            'IsolationForest': iso_labels,
            'SGDOneClassSVM': svm_labels
        }, index=data.index)

        # 상관관계 기반 이상 탐지 결과 추가
        results = pd.concat([results, pca_outliers, range_outliers], axis=1)

        # 최종 다수결: 모든 탐지 결과의 평균이 0.5 이상이면 이상치로 판별
        results['faultNumber'] = (results.mean(axis=1) >= 0.5).astype(int)

        return results[['faultNumber']]


In [8]:
if __name__ == "__main__":
    # 학습 데이터 준비
    np.random.seed(42)
    train_data = train_df

    # 모델 생성 및 학습
    model = HybridAnomalyDetector()
    model.fit(train_data)

    # 새로운 데이터에 대한 예측
    new_data = test_df
    predictions = model.predict(new_data)


제거된 피처: ['xmeas_13', 'xmeas_16', 'xmeas_19', 'xmv_3', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_11']
상관관계 높은 피처 그룹: [['xmeas_7'], ['xmeas_7', 'xmeas_13'], ['xmeas_18'], ['xmeas_1'], ['xmeas_10'], ['xmeas_12'], ['xmeas_15'], ['xmeas_18', 'xmeas_19'], ['xmeas_17']]
상관관계가 낮은 피처: ['xmeas_5', 'xmeas_6', 'xmeas_14', 'xmeas_24', 'xmeas_26', 'xmeas_28', 'xmeas_32', 'xmeas_37', 'xmeas_39', 'xmeas_40', 'xmeas_41', 'xmv_4']


In [10]:
predictions

,Final_Label
0,0
1,0
2,0
3,0
4,0
...,...
710395,0
710396,0
710397,0
710398,0


In [ ]:
p

In [9]:
predictions.to_csv('multi_2.csv')